In [0]:
from google.colab import drive  #connecting drive account
drive.mount('/content/drive')

IMPORT LIBRARIES

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0
!pip install tensorflow==2.0.0

In [0]:
import numpy as np
import os
import PIL
import pickle

IMPORT DATASETS AND NORMALIZE IT

In [0]:
traffic_signs_data_folder='/content/drive/My Drive/Colab Notebooks/traffic-signs-data/' #dataset
training_file = traffic_signs_data_folder + 'train.p'
validation_file= traffic_signs_data_folder + 'valid.p'
testing_file = traffic_signs_data_folder + 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

In [0]:
X_train, y_train = train['features'], train['labels'] #inputs and outputs
X_validation, y_validation = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
X_train.shape #32x32 colorful image

(34799, 32, 32, 3)

In [0]:
X_test.shape

(12630, 32, 32, 3)

In [0]:
from sklearn.utils import shuffle 
X_train, y_train = shuffle(X_train, y_train)

In [0]:
X_train_gray_norm = (X_train - 255)/255  #normalizing our data
X_test_gray_norm = (X_test - 255)/255
X_validation_gray_norm = (X_validation - 255)/255

In [0]:
X_train_gray_norm

In [0]:
y_validation.shape

MODEL TRAINING

In [0]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

LeNet = models.Sequential()

LeNet.add(layers.Conv2D(6, (5,5), activation = 'relu', input_shape = (32,32,3)))
LeNet.add(layers.AveragePooling2D())


LeNet.add(layers.Conv2D(16, (5,5), activation = 'relu'))
LeNet.add(layers.AveragePooling2D())

LeNet.add(layers.Flatten())

LeNet.add(layers.Dense(120, activation = 'relu'))

LeNet.add(layers.Dense(84, activation = 'relu'))

LeNet.add(layers.Dense(43, activation = 'softmax'))
LeNet.summary()

In [0]:
LeNet.compile(optimizer = 'Adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [0]:
history = LeNet.fit(X_train_gray_norm,
                 y_train, 
                 batch_size = 500,
                 nb_epoch = 50,
                 verbose = 1, #verbose-ayrıntılı
                 validation_data = (X_validation_gray_norm, y_validation))

MODEL EVALUATION

In [0]:
score = LeNet.evaluate(X_test_gray_norm, y_test)
print('Test Accuracy: {}'.format(score[1])) #success rate

12630/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [0]:
# Create a converter
from tensorflow import lite

converter = tf.lite.TFLiteConverter.from_keras_model(LeNet)
# Convert the model
tflite_model = converter.convert()
# Create the tflite model file
tflite_model_name = "model.tflite"
open(tflite_model_name, "wb").write(tflite_model)

262040